# Demo Notebook for Bayesian Hyperparameter Searches

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [ ]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
dj.config['display.limit'] = 200

name = 'iclr'
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [ ]:
import re
import torch
import numpy as np
import pickle 
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)
from collections import OrderedDict, Iterable
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from nnfabrik.main import *
from nnfabrik.utility.hypersearch import Bayesian
from nnsysident.tables.bayesian import *

# Dataset

In [ ]:
paths = ['data/static22564-2-12-preproc0.zip',
         'data/static22564-2-13-preproc0.zip',
         'data/static22564-3-8-preproc0.zip',
         'data/static22564-3-12-preproc0.zip']
dataset_fn = 'nnsysident.datasets.mouse_loaders.static_shared_loaders'

# define fixed parameters
dataset_config = dict(
    paths=paths,
    batch_size=64,
    multi_match_n=neuron_n,
    multi_match_base_seed=1,
    image_n=image_n,
    image_base_seed=1
)

# define parameters to optimize
dataset_config_auto = dict()

# Model

In [ ]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'

# define fixed parameters
model_config = {"init_mu_range": 0.55,
                "init_sigma": 0.4,
                'input_kern': 15,
                'hidden_kern': 13,
                'gamma_input': 1.,
                'grid_mean_predictor': {'type': 'cortex', 'input_dimensions': 2, 'hidden_layers': 0, 'hidden_features': 0, 'final_tanh': False},
}

# define parameters to optimize
model_config_auto = dict(
    gamma_readout={"type": "range", "bounds": [1e-5, 1e3], "log_scale": True},
    share_features={"type": "choice", "values": [True, False]},
    share_transform={"type": "choice", "values": [True, False]},
)

# Trainer

In [ ]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'

# define fixed parameters
trainer_config = dict(detach_core=False)

# define parameters to optimize
trainer_config_auto = dict()

# Run Bayesian Search

In [ ]:
fabrikant_name = "kklurz"
total_trials = 100
trained_model_table = 'nnsysident.tables.bayesian.TrainedModelBayesian'

autobayes = Bayesian(dataset_fn, dataset_config, dataset_config_auto,
                     model_fn, model_config, model_config_auto,
                     trainer_fn, trainer_config, trainer_config_auto, architect=fabrikant_name,
                     trained_model_table=trained_model_table, total_trials=total_trials)

best_parameters, _, _, _ = autobayes.run()

### Optional: add optimized configs to their tables

In [ ]:
model_config.update(best_parameters['model'])
dataset_config.update(best_parameters['dataset'])
trainer_config.update(best_parameters['trainer'])

Model().add_entry(model_fn=model_fn,
                  model_config=model_config,
                  model_fabrikant=fabrikant_name,
                  skip_duplicates=True)

Dataset().add_entry(dataset_fn=dataset_fn,
                  dataset_config=dataset_config,
                  dataset_fabrikant=fabrikant_name,
                  skip_duplicates=True)

Trainer().add_entry(trainer_fn=trainer_fn,
                  trainer_config=trainer_config,
                  trainer_fabrikant=fabrikant_name,
                  skip_duplicates=True)